In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pneumo import dataset
from torch.utils.data import DataLoader
import os
import glob
import albumentations as A

In [3]:
IMG_PATH = "/home/wilmer-linux/Projects/data/pneumo_s1/"
LABEL_PATH = os.path.join(IMG_PATH, '../stage_2_train.csv')

fns = glob.glob(os.path.join(IMG_PATH, "**/*.dcm"), recursive=True)

In [4]:
from sklearn.model_selection import train_test_split

train_fns, valid_fns = train_test_split(fns, train_size=0.8, random_state=42, shuffle=True)
print(len(train_fns), len(valid_fns))

12235 3059


In [5]:
config = {
    "lr" : 1e-3,
    "batch_size": 16,
}

In [6]:
from albumentations import (
    Compose, HorizontalFlip,
    RandomBrightness, RandomContrast, RandomGamma,OneOf,
    ShiftScaleRotate,GridDistortion, ElasticTransform, JpegCompression, HueSaturationValue,
    RandomBrightness, RandomContrast, Blur, MotionBlur, MedianBlur, GaussNoise,CenterCrop,
    IAAAdditiveGaussianNoise,GaussNoise,OpticalDistortion,RandomSizedCrop, Resize, RandomBrightnessContrast, ToFloat
)

IMG_SIZE= (256, 256)

TRAIN_AUGS = Compose([
    ToFloat(max_value=1),
    Resize(IMG_SIZE[0], IMG_SIZE[1]),
    HorizontalFlip(p=0.3),
    ShiftScaleRotate(p=0.1, rotate_limit=30),
    OneOf([
        RandomGamma((90,110)),
        RandomBrightnessContrast(0.1, 0.1),
         ], p=0.2),
],p=1)

VALID_AUGS = Compose([
    ToFloat(max_value=1),
    Resize(IMG_SIZE[0], IMG_SIZE[1])
])

In [7]:
train_ds = dataset.PneumoDataset(LABEL_PATH, train_fns, transforms=TRAIN_AUGS)
valid_ds = dataset.PneumoDataset(LABEL_PATH, valid_fns, transforms=VALID_AUGS)

train_loader = DataLoader(train_ds, batch_size=config["batch_size"], shuffle=True, num_workers=3, multiprocessing_context='spawn')
valid_loader = DataLoader(valid_ds, batch_size=config["batch_size"], shuffle=False, num_workers=0)

In [8]:
from pneumo.models import UNetLightning, MetaUNetLightning
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint
from pytorch_lightning.loggers import WandbLogger

model_name = "unet++_effb2_dropout0_mixedwslogdiceloss_nobce_metaencoder"
ckpt_path = "/home/wilmer-linux/Projects/code/pneumo/ckpt/"

unet = MetaUNetLightning(learning_rate=config["lr"], smoothing=0.1).cuda()
wandb_logger = WandbLogger(project="pnuemo_2", name=model_name)
es = EarlyStopping(monitor="valid_loss", min_delta=0.001, patience=20)
checkpoint = ModelCheckpoint(dirpath=ckpt_path, filename=model_name, monitor="valid_dice_coeff", save_top_k=1)

trainer = Trainer(
    gpus=1,
    auto_select_gpus=True, 
    precision=16,
    logger=wandb_logger,
    callbacks=[es, checkpoint]
)

Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [ ]:
trainer.fit(unet, train_dataloader=train_loader, val_dataloaders=valid_loader)

/home/wilmer-linux/miniconda3/envs/faceai-nada/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:730: LightningDeprecationWarning: `trainer.fit(train_dataloader)` is deprecated in v1.4 and will be removed in v1.6. Use `trainer.fit(train_dataloaders)` instead. HINT: added 's'
  rank_zero_deprecation(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
wandb: Currently logged in as: wilmer (use `wandb login --relogin` to force relogin)



  | Name         | Type                | Params
-----------------------------------------------------
0 | encoder      | EfficientNetEncoder | 7.7 M 
1 | decoder      | UnetPlusPlusDecoder | 2.7 M 
2 | meta_encoder | MetaEncoder         | 57    
3 | seg_head     | Sequential          | 625   
-----------------------------------------------------
10.4 M    Trainable params
0         Non-trainable params
10.4 M    Total params
20.803    Total estimated model params size (MB)
/home/wilmer-linux/miniconda3/envs/faceai-nada/lib/python3.9/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:617: UserWarning: Checkpoint directory /home/wilmer-linux/Projects/code/pneumo/ckpt exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")


Validation sanity check: 0it [00:00, ?it/s]

/home/wilmer-linux/miniconda3/envs/faceai-nada/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:110: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 6 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

In [ ]:
# optimize threshold for dice coeff